# Neural Networks Experiments

## Estudiantes: <br>
Tomás Rojas <br>
Matías Montagna <br>
Alonso Utreras

## The objective of this notebook is to show the results from different models, including different inputs, but trying to get the same output.

In [ ]:
# Get data
import pandas as pd
import torch
import torch.optim as optim
import numpy as np
from sklearn.model_selection import train_test_split

all_data = pd.read_csv(".\data_product_temp_mp25")

In [ ]:
all_data = all_data.dropna()
X = all_data[all_data.columns[-4]]
Y = all_data[all_data.columns[-2]]
# Y = all_data[all_data.columns[:-4]].join(Y) # para que tenga todo



In [ ]:
all_data.head()

In [ ]:
X.head()

In [ ]:
Y.head()

### Dividing data into train, test and validation sets
We chose a train size of 70% of all data, while 15% corresponds to test and 15% to validation data.

In [ ]:
test_data_size = int(len(all_data) * 0.7)

train_data = all_data[:-test_data_size]
test_data = all_data[-test_data_size:]

# We can do this if sklearn works fine with DataFrames
X_train, X_2, y_train, y_2 = train_test_split(X, Y, test_size=0.3, random_state=42)
X_test, X_val, y_test, y_val = train_test_split(X_2, y_2, test_size=0.3, random_state=42)



In [ ]:
x_train_np = np.asarray(X_train.values)


y_train_np = np.asarray(y_train.values)
y_train_np

In [ ]:
# device = ('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
print(device)

X_train = torch.from_numpy(x_train_np).to(device)
y_train = torch.from_numpy(y_train_np).to(device)

### Normalizing train data:
We read here https://stackabuse.com/time-series-prediction-using-lstm-with-pytorch-in-python/
that it is important to normalize data when working with time series. 



In [ ]:
from sklearn.preprocessing import MinMaxScaler


scaler = MinMaxScaler(feature_range=(-1, 1))
train_data_normalized = scaler.fit_transform(train_data.reshape(-1, 1))

# Transforming data into tensors
train_data_normalized = torch.FloatTensor(train_data_normalized).view(-1)


## Classes for training our models:

## First model, which is a simple ff NN using just date.
Simple feedforward NN
Input: Date

In [ ]:
import models as m

# a = m.SimpleFCModel(1, 1)

modelo_1_temp_pm = torch.nn.Sequential(
    torch.nn.Linear(1, 5),
    torch.nn.ReLU(),
    torch.nn.Linear(5, 1),
).to(device)

loss_1 = torch.nn.L1Loss(reduction='mean')




## Training function

In [ ]:
import time

# TODO: Completar
def batcher(training_data, batch_size=365*24):
    """This generates our batches. Given the  """
    inout_seq = []
    L = len(training_data)
    for i in range(L-batch_size):
        train_seq = input_data[i:i+batch_size]
        train_label = input_data[i+batch_size:i+batch_size+1]
        inout_seq.append((train_seq ,train_label))
    return inout_seq


def train(model, x_train, y_train, optimizer, loss_function, epochs=5, batch_size=365*24):
    model.train()
    total_loss = 0
    for i in range(epochs):
        for x_i, y_i in zip(x_train, y_train):
            optimizer.zero_grad()
            y_pred = model(x_i)

            loss = loss_function(y_pred, y_train)
            total_loss += loss.item()
            loss.backward()
            optimizer.step()

        print(f'epoch: {i} loss: {loss.item():10.8f}')
    
    return total_loss

def epoch_time(start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs

def test(model, x_test, y_test, loss_function, batch_size=365*24):
    model.eval()
    total_loss = 0

    with torch.no_grad():
        for x_i, y_i in zip(x_test, y_test):
            # predict data using the given model
            prediction = model(x_i)
            # Compute loss
            total_loss += loss_function(prediction, y_i).item()

    print(total_loss)
    return total_loss


### Execute Training and Testing:

In [ ]:
# Hyperparamters
n_epochs = 3


In [ ]:
# Execute training
def execute_training(model, x_train, y_train, x_test, y_test, optimizer, loss_function,  n_epochs=5, batch_size=365*24):
    # Train
    best_valid_loss = float('inf')

    start_time = time.time()
    train_loss = train(model, x_train, y_train, optimizer, loss_function, n_epochs, batch_size)
    end_time = time.time();
    train_time = end_time - start_time

    print(f'Training time = {train_time}')
    print(f'Train Loss: {train_loss}')

    # # Test
    start_time = time.time()

    valid_loss = test(model, x_test, y_test, loss_function, batch_size)
    end_time = time.time()
    test_time = end_time - start_time 

    print(f'\t Val. Loss: {valid_loss:.3f}')

    # Save results from the best trained model
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        # torch.save(model.state_dict())


In [ ]:

# # Load the model with best results in the validation dataset
# model.load_state_dict(torch.load('{}.pt'.format(model_name)))

# Limpiar ram de cuda
torch.cuda.empty_cache()

## TODO: Plot predictions v/s real data

In [ ]:
model = modelo_1_temp_pm
loss = loss_1
# optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
optimizer = optim.Adam(model.parameters())

In [ ]:
# X_train = X_train.transpose(0, 1)
# y_train = y_train.transpose(0, 1)
X_tensor = X_train.view(-1, 1)
y_tensor = y_train.view(-1, 1)
# X_tensor = X_tensor.transpose(0, 1)
# y_tensor = y_tensor.transpose(0, 1)

In [ ]:
X_train = X_tensor[0:40000]
y_train = y_tensor[0:40000]
X_test = X_tensor[40000:41000]
y_test = y_tensor[40000:41000]

In [ ]:
execute_training(model.double(), X_train, y_train, X_test, y_test, optimizer, loss, n_epochs=5)